In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import from_json, col, when
# from pyspark.sql.types import StructType, StringType, DoubleType

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, window, expr, broadcast
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("flights_stream") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.4.1") \
    .getOrCreate()

# Kafka source
kafka_df = (spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "flights.events")
    .option("startingOffsets", "latest")
    .load())

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/13 16:04:49 WARN Utils: Your hostname, DESKTOP-VS2UPJ4, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/13 16:04:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/anhtu77/miniconda3/envs/datalab/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/anhtu77/.ivy2.5.2/cache
The jars for the packages stored in: /home/anhtu77/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-295b4bc5-4b11-446e-8e17-f03797b82de2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 

Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.classic.SparkSession.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(Constru

In [ ]:
# define schema for flights event (select columns you need)
flights_schema = StructType() \
    .add("YEAR", IntegerType()) \
    .add("MONTH", IntegerType()) \
    .add("DAY", IntegerType()) \
    .add("DAY_OF_WEEK", IntegerType()) \
    .add("AIRLINE", StringType()) \
    .add("FLIGHT_NUMBER", IntegerType()) \
    .add("TAIL_NUMBER", StringType()) \
    .add("ORIGIN_AIRPORT", StringType()) \
    .add("DESTINATION_AIRPORT", StringType()) \
    .add("SCHEDULED_DEPARTURE", IntegerType()) \
    .add("DEPARTURE_TIME", IntegerType()) \
    .add("DEPARTURE_DELAY", IntegerType()) \
    .add("TAXI_OUT", IntegerType()) \
    .add("WHEELS_OFF", IntegerType()) \
    .add("SCHEDULED_TIME", IntegerType()) \
    .add("ELAPSED_TIME", IntegerType()) \
    .add("AIR_TIME", IntegerType()) \
    .add("DISTANCE", IntegerType()) \
    .add("WHEELS_ON", IntegerType()) \
    .add("TAXI_IN", IntegerType()) \
    .add("SCHEDULED_ARRIVAL", IntegerType()) \
    .add("ARRIVAL_TIME", IntegerType()) \
    .add("ARRIVAL_DELAY", IntegerType()) \
    .add("DIVERTED", IntegerType()) \
    .add("CANCELLED", IntegerType()) \
    .add("CANCELLATION_REASON", StringType()) \
    .add("AIR_SYSTEM_DELAY", IntegerType()) \
    .add("SECURITY_DELAY", IntegerType()) \
    .add("AIRLINE_DELAY", IntegerType()) \
    .add("LATE_AIRCRAFT_DELAY", IntegerType()) \
    .add("WEATHER_DELAY", IntegerType())
    # add other fields as needed

parsed = kafka_df.select(from_json(col("value").cast("string"), flights_schema).alias("data")).select("data.*")
# cast event_time properly
events = parsed.withColumn("event_time", to_timestamp("event_time"))

# read lookups (small) and broadcast
airlines = spark.read.csv("data/airlines.csv", header=True)  # or read from Cassandra
airports = spark.read.csv("data/airports.csv", header=True)

# Example: 30-minute tumbling window average arrival delay per airline
agg = (events
       .withWatermark("event_time", "1 hour")
       .groupBy(window(col("event_time"), "30 minutes"), col("OP_CARRIER"))
       .agg(
            expr("avg(cast(ARR_DELAY as double)) as avg_arr_delay"),
            expr("count(*) as num_flights")
       ))

# enrich with airline names using broadcast
agg_enriched = agg.join(broadcast(airlines), agg.OP_CARRIER == airlines.Code, "left") \
                  .select("window", "OP_CARRIER", "Name", "avg_arr_delay", "num_flights")



In [ ]:
df_parsed = df_raw.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")

In [ ]:
df_transformed = df_parsed.withColumn(
    "delay_category",
    when(col("delay") > 0, "Delayed")
    .otherwise("On Time")
)

In [ ]:
hdfs_query = df_transformed.writeStream \
    .format("parquet") \
    .option("path", "hdfs://hadoop-namenode:9000/flights/stream_output") \
    .option("checkpointLocation", "/tmp/hdfs_checkpoint") \
    .outputMode("append") \
    .start()